<a href="https://colab.research.google.com/github/adolfoguimaraes/inteligenciaartificial/blob/main/code/05_Prolog_Parte2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prolog com Python - Parte 2

Nesse notebook vamos explorar mais algunas conceitos de Prolog. 

As bases utilizadas neste documento estão disponíveis no repositório: https://github.com/adolfoguimaraes/inteligenciaartificial. 

Para facilitar o uso dessas bases aqui no Colab, essa base está replicada também no diretório https://drive.google.com/drive/folders/19euiVFFa32TmqR_WAE8Xer-8G273efXN?usp=sharing. 

In [3]:
# Para uso no colab
dataset_path = "" 

In [2]:
# Para uso no github
dataset_path = "../datasets/kb_prolog/"

Para cada notebook, devemos fazer a instalação dos pacotes necessários novamente.

In [ ]:
# Rodar essa célular apenas se tiver rodando no colab.

!sudo apt install swi-prolog
!pip install pyswip

In [5]:
# Carregar a biblioteca 
from pyswip import Prolog

# Instanciar o objeto
prolog = Prolog()

In [6]:
## Método para consultar a base
def consultar_base(pergunta: str):

  """
    Esse método recebe uma pergunta com parâmetro e retorna True se assertiva 
    for verdadeira e False, caso contrário. Caso a pergunta seja com variáveis, 
    o método vai retornar uma lista de dicionários, onde a chave de cada 
    dicionário é uma variável passada. 

    :param pergunta: str
    
    :return : bool, list
  """

  result_ = prolog.query(pergunta)
  result_list = list(result_)

  if len(result_list) == 0: return False
  else:
    if len(result_list[0].keys()) == 0: return True
    else:
      return result_list


Terminamos nossa última atividade vendo como podemos trablhar com recursão em Prolog. Vamos ver mais alguns exemplos. 

## Mapeando um Grafo

Podemos mapear um grafo como um problema de prolog. Vamos considerar o seguinte problema: 

O grafo a seguir representa um mapa cujas cidades são representadas por letras e cujas estradas (de sentido único) são representados por números que indicam sua extensão em quilômetros. Vamos usar essa imagem para criar nossa base de conhecimento a partir do predicado **estrada(Origem, Destino, km)**.

Clique [aqui](https://i.ibb.co/Nm4d238/Captura-de-Tela-2021-10-22-a-s-15-14-40.png) para visualizar o grafo.

Carregue o arquivo `base3.pl` do repositório com o mapeamento do grafo da imagem acima. 

In [7]:
# Carregar a base3.pl
prolog.consult(dataset_path + "base3.pl")

Faça algumas consultas para testar a base. 

In [ ]:
# Insira suas consultas neste espaço

Vamos atualizar a base com um regra que permita verificar se duas cidades são adjacentes: 

```prolog
sao_adjacentes(X, Y) :- estrada(X, Y, _) ; estrada(Y, X,_).
```
Atualize o arquivo e faça algumas consultas para restar a regra.

In [8]:
prolog.consult(dataset_path + "base3.pl")

In [9]:
consultar_base("sao_adjacentes(a, d)")

True

In [10]:
consultar_base("sao_adjacentes(b, d)")

False

In [11]:
consultar_base("sao_adjacentes(c, X)")

[{'X': 'd'}, {'X': 'f'}, {'X': 'b'}]

Ótimo. Nossa base está funcionando 😀. 

Vamos criar uma regra mais interessante. Crie um regra que retorne todas as cidades que estão a uma distância menor que X de uma outra cidade. 

É importante ressaltar que quando queremos passar alguma informação para máquina de inferência devemos utiliar as variáveis das regras para isso. A regra abaixo faz o que se pede. Vamos analisa-la e testa-la. 

```prolog
proximo(X, Y, DIST) :- estrada(X, Y, Z) , Z =< DIST.
```



In [12]:
prolog.consult(dataset_path + "base3.pl")

Vamos fazer a seguinte pergunta: **Quais são as cidades que estão a menos de 25km de distância?**

Isso pode ser traduzido em prolog na seguinte pergunta: 

```
proximo(A, B, 25).
```

In [13]:
consultar_base("proximo(A, B, 25)")

[{'A': 'a', 'B': 'b'},
 {'A': 'a', 'B': 'd'},
 {'A': 'b', 'B': 'c'},
 {'A': 'c', 'B': 'd'}]

## Agora é sua vez 

Crie uma regra que verifique se existe um caminho entre duas cidades. 

Atualize o arquivo base3.pl com as nova regra e teste-a.



In [ ]:
#Faça suas consultas a partir deste espaço

## Listas em Prolog

Para fecharmos nosso estudo de Prolog, vamos trabalhar com uma estrutura muito importante: as listas. 

O Prolog  "enxerga" umas lista em duas partes: `head` (cabeça) e `tail` (calda). 

Por exemplo, na lista `[1,2,3,4]`, `head = 1` e `tail = [2,3,4]`. Observe que a cabeça é sempre um elemento da lista e a calda uma lista de elementos. 

Para entender melhor, vamos trabalhar com o arquivo `base4.pl`.

In [16]:
prolog.consult(dataset_path + "base4.pl")

Com a primeira regra podemos verificar se um elemento pertence a lista. Ela funciona da seguinte forma. 

A regra: 

```prolog
esta_na_lista([X|T], X).
```

Vai retornar verdadeira se o elemento pesquisado é a cabeça da lista. Se essa regra não retornar verdadeira, será testada a segunda regra com o mesmo nome e assim por diante. 

No nosso caso, a regra: 

```prolog
esta_na_lista([_|T],X) :- esta_na_lista(T, X).
```
Observe que essa regra chama recursivamente a mesma regra passando como lista a calda. Ou seja, todos os elementos menos o primeiro que já foi testado. Enquanto o elemento procurado não aparecer na primeira posição, essa recursão vai ser chamada. Quando não tiver mais elementos, a recursão vai parar. 

Execute algumas consultas para testar a regra. 


In [17]:
consultar_base("esta_na_lista([1,2,3],2)")

True

In [18]:
consultar_base("esta_na_lista([1,2,3],4)")

False

Se realizarmos a busca com uma variável, a consulta vai retornar a lista de elementos da lista. 

In [19]:
consultar_base("esta_na_lista([1,2,3],X)")

[{'X': 1}, {'X': 2}, {'X': 3}]

**Vamos para mais um exemplo**

A segunda regra retorna `Verdadeiro` se a primeira lista é composta só por `a's` e a segunda só por `b's`. Analise a regra e, com base no que vimos, tente entendê-la. Execute algumas consultas para testar. 

In [20]:
prolog.consult(dataset_path + "base4.pl")

In [21]:
consultar_base("a2b([a,a,a],[b,b,b])")

True

In [22]:
consultar_base("a2b([a,b,a],[b,b,b])")

False